In [1]:
## importing needed library

In [3]:
import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm

from PIL import Image
from tensorflow.keras.utils import to_categorical

import seaborn as sns
import matplotlib.image as img

In [4]:
print(tfa.__version__)

0.17.1


In [4]:
train_csv=pd.read_csv("Human Action Recognition data/Training_set.csv")
test_csv=pd.read_csv("Human Action Recognition data/Testing_set.csv")

In [5]:
train_fol=glob.glob('Human Action Recognition data/Train/*')
test_fol=glob.glob('Human Action Recognition data/Test/*')

In [6]:
len(train_fol)

12601

In [7]:
filename= train_csv['filename']
action=train_csv['label']

In [8]:
img_data=[]
img_label=[]
lenght=len(train_fol)
for i in range(len(train_fol)-1):
    t='Human Action Recognition data/Train/' + filename[i]
    temp_img=Image.open(t)
    img_data.append(np.asarray(temp_img.resize((160,160))))
    img_label.append(action[i])

In [11]:
img_dta

array([[[[234, 161,  89],
         [231, 164,  81],
         [217, 157,  83],
         ...,
         [ 39,   2,   0],
         [ 35,   0,   4],
         [ 34,   0,  11]],

        [[222, 159,  84],
         [222, 162,  83],
         [218, 165, 100],
         ...,
         [ 37,   3,   1],
         [ 33,   0,   1],
         [ 31,   0,   4]],

        [[223, 172,  98],
         [225, 177, 102],
         [228, 187, 132],
         ...,
         [ 33,   3,   3],
         [ 31,   1,   0],
         [ 30,   1,   0]],

        ...,

        [[108, 153, 196],
         [116, 163, 201],
         [116, 165, 206],
         ...,
         [196, 146, 111],
         [194, 144, 109],
         [193, 143, 108]],

        [[104, 149, 192],
         [111, 160, 196],
         [109, 161, 202],
         ...,
         [195, 145, 110],
         [194, 144, 109],
         [192, 142, 107]],

        [[106, 151, 195],
         [108, 157, 194],
         [101, 153, 194],
         ...,
         [195, 145, 110],
        

In [10]:
img_dta = img_data
img_dta = np.asarray(img_dta)
y_train = to_categorical(np.asarray(train_csv['label'].factorize()[0]))
print(y_train[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [23]:
vgg_model = Sequential()

trained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(160,160,3),
                   pooling='avg',classes=15,
                   weights='imagenet',classifier_activation='softmax')

for layer in trained_model.layers:
        layer.trainable=False

vgg_model.add(trained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(15, activation='softmax'))

In [24]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
vgg_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 15)                7695      
                                                                 
Total params: 14,985,039
Trainable params: 270,351
Non-trainable params: 14,714,688
_________________________________________________________________


In [27]:
history = vgg_model.fit(img_dta,y_train, epochs=25)

Epoch 1/25
394/394 [==============================] - 1966s 5s/step - loss: 2.0347 - accuracy: 0.4556
Epoch 2/25
394/394 [==============================] - 1895s 5s/step - loss: 1.2086 - accuracy: 0.6043
Epoch 3/25
394/394 [==============================] - 1876s 5s/step - loss: 0.9444 - accuracy: 0.6898
Epoch 4/25
394/394 [==============================] - 1852s 5s/step - loss: 0.7302 - accuracy: 0.7611
Epoch 5/25
394/394 [==============================] - 1792s 5s/step - loss: 0.5534 - accuracy: 0.8159
Epoch 6/25
394/394 [==============================] - 1792s 5s/step - loss: 0.4017 - accuracy: 0.8658
Epoch 7/25
394/394 [==============================] - 1798s 5s/step - loss: 0.2730 - accuracy: 0.9160
Epoch 8/25
394/394 [==============================] - 1805s 5s/step - loss: 0.2114 - accuracy: 0.9329
Epoch 9/25
394/394 [==============================] - 1809s 5s/step - loss: 0.1627 - accuracy: 0.9502
Epoch 10/25
394/394 [==============================] - 1818s 5s/step - loss: 0.156

In [35]:
#saving


vgg16_path= os.path.join('','model_weight')
os.makedirs(vgg16_path,exist_ok=True)

with open(os.path.join(vgg16_path,'model.json'), 'w') as f:
    f.write(vgg_model.to_json())
    
vgg_model.save_weights(os.path.join(vgg16_path, 'vgg_model_weight.h5'))

vgg_model.save(os.path.join(vgg16_path,'full_model.h5'))

In [ ]:
vgg_model.predict()

In [42]:
filename_test=test_csv['filename']
a=random.randint(1,100)
img_test=[]
t='Human Action Recognition data/test/' + filename_test[a]
temp_img=Image.open(t)
img_test.append(np.asarray(temp_img.resize((160,160))))
    


In [44]:
img_test,a

([array([[[236, 240, 239],
          [236, 240, 239],
          [237, 240, 239],
          ...,
          [220, 216, 205],
          [221, 217, 206],
          [221, 217, 206]],
  
         [[245, 246, 245],
          [244, 245, 244],
          [244, 246, 245],
          ...,
          [219, 215, 204],
          [221, 217, 206],
          [221, 217, 206]],
  
         [[240, 240, 237],
          [247, 246, 244],
          [251, 250, 248],
          ...,
          [219, 215, 204],
          [220, 216, 205],
          [220, 216, 205]],
  
         ...,
  
         [[227, 229, 226],
          [226, 228, 225],
          [223, 225, 222],
          ...,
          [213, 209, 200],
          [214, 210, 201],
          [214, 210, 201]],
  
         [[228, 230, 227],
          [226, 228, 225],
          [224, 226, 223],
          ...,
          [214, 210, 201],
          [215, 211, 202],
          [215, 211, 202]],
  
         [[228, 230, 227],
          [227, 229, 226],
          [224, 226, 223

In [48]:
vgg_model.predict(np.asanyarray(img_test))

1/1 [==============================] - 0s 316ms/step


array([[6.5076696e-05, 1.4983140e-02, 3.3593488e-01, 3.0306046e-05,
        4.9805936e-07, 8.0662547e-04, 5.6546125e-11, 1.7190027e-11,
        9.4573991e-03, 5.6058012e-05, 1.6837452e-08, 9.9781463e-03,
        4.3292038e-02, 2.6626176e-10, 5.8539575e-01]], dtype=float32)

In [51]:
vg16=Sequential()

In [55]:
from keras.models import load_model
model = load_model("model_weight/full_model.h5")

In [56]:
model.predict(np.asanyarray(img_test))

1/1 [==============================] - 0s 272ms/step


array([[6.5076696e-05, 1.4983140e-02, 3.3593488e-01, 3.0306046e-05,
        4.9805936e-07, 8.0662547e-04, 5.6546125e-11, 1.7190027e-11,
        9.4573991e-03, 5.6058012e-05, 1.6837452e-08, 9.9781463e-03,
        4.3292038e-02, 2.6626176e-10, 5.8539575e-01]], dtype=float32)